In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet
/kaggle/input/bengaliai-cv19/train_multi_diacritics.csv
/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/class_map_corrected.csv
/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet


In [2]:
from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns

Using TensorFlow backend.


In [3]:
train=pd.read_csv('/kaggle/input/bengaliai-cv19/train.csv')
test=pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')

In [4]:
sample_sub = pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')

In [5]:
train.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [6]:
test.head()

,row_id,image_id,component
0,Test_0_consonant_diacritic,Test_0,consonant_diacritic
1,Test_0_grapheme_root,Test_0,grapheme_root
2,Test_0_vowel_diacritic,Test_0,vowel_diacritic
3,Test_1_consonant_diacritic,Test_1,consonant_diacritic
4,Test_1_grapheme_root,Test_1,grapheme_root


In [7]:
sample_sub.head()

,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,0
2,Test_0_vowel_diacritic,0
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,0


In [8]:
class_map=pd.read_csv('/kaggle/input/bengaliai-cv19/class_map_corrected.csv')
class_map.head()

,component_type,label,component
0,grapheme_root,0,ং
1,grapheme_root,1,ঃ
2,grapheme_root,2,অ
3,grapheme_root,3,আ
4,grapheme_root,4,ই


In [9]:
train.shape

(200840, 5)

In [10]:
train.drop(['grapheme'],axis=1,inplace=False)

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic
0,Train_0,15,9,5
1,Train_1,159,0,0
2,Train_2,22,3,5
3,Train_3,53,2,2
4,Train_4,71,9,5
...,...,...,...,...
200835,Train_200835,22,7,2
200836,Train_200836,65,9,0
200837,Train_200837,2,1,4
200838,Train_200838,152,9,0


In [11]:
print(f'Number of unique grapheme roots: {train["grapheme_root"].nunique()}')
print(f'Number of unique vowel diacritic: {train["vowel_diacritic"].nunique()}')
print(f'Number of unique consonant diacritic: {train["consonant_diacritic"].nunique()}')

Number of unique grapheme roots: 168
Number of unique vowel diacritic: 11
Number of unique consonant diacritic: 7


In [12]:
img_size=64
channels=1

In [53]:
train_df = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_3.parquet',engine='pyarrow')
X_train = train_df.drop(['image_id'], axis=1)



In [54]:
X_train=np.array(X_train).reshape(-1,137,236)

In [55]:
X_train.shape

(50210, 137, 236)

In [16]:
train.drop(columns='image_id',inplace=True)

In [56]:
x=X_train.shape[0]

In [18]:
train.head()

,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,15,9,5,ক্ট্রো
1,159,0,0,হ
2,22,3,5,খ্রী
3,53,2,2,র্টি
4,71,9,5,থ্রো


In [57]:
Y_grapheme=train.iloc[3*x:4*x,0]
Y_vowel=train.iloc[3*x:4*x,1]
Y_consonant=train.iloc[3*x:4*x,2]

In [58]:
import cv2
import gc
train_images=[]
for i in range(x):
    image=X_train[i]
    image=cv2.resize(image,(img_size,img_size),interpolation=cv2.INTER_AREA)
    train_images.append(image)
del X_train
gc.collect()
    

104

In [36]:
inputs=Input(shape=(img_size,img_size,1))

In [22]:
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=(64, 64, 1))(inputs)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256,kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization()(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dense(1024, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(512, activation = "relu")(model)

head_root = Dense(168, activation = 'softmax')(dense)
head_vowel = Dense(11, activation = 'softmax')(dense)
head_consonant = Dense(7, activation = 'softmax')(dense)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 64)   36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 64, 64, 64)   36928       conv2d_2[0][0]                   
____________________________________________________________________________________________

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
train_images=np.array(train_images).reshape(-1,64,64,1)

In [60]:
from keras.utils import to_categorical
Y_grapheme=to_categorical(Y_grapheme,num_classes=168)
Y_vowel=to_categorical(Y_vowel,num_classes=11)
Y_consonant=to_categorical(Y_consonant,num_classes=7)

In [61]:
model.fit(train_images,[Y_grapheme,Y_vowel,Y_consonant],batch_size=32,epochs=10,validation_split=0.1)

Train on 45189 samples, validate on 5021 samples
Epoch 1/10
45189/45189 [==============================] - 77s 2ms/step - loss: 0.6440 - dense_3_loss: 0.4231 - dense_4_loss: 0.1148 - dense_5_loss: 0.1061 - dense_3_accuracy: 0.8925 - dense_4_accuracy: 0.9726 - dense_5_accuracy: 0.9719 - val_loss: 1.7808 - val_dense_3_loss: 0.8474 - val_dense_4_loss: 0.7347 - val_dense_5_loss: 0.1984 - val_dense_3_accuracy: 0.8068 - val_dense_4_accuracy: 0.7853 - val_dense_5_accuracy: 0.9530
Epoch 2/10
45189/45189 [==============================] - 77s 2ms/step - loss: 0.4886 - dense_3_loss: 0.3237 - dense_4_loss: 0.0863 - dense_5_loss: 0.0785 - dense_3_accuracy: 0.9131 - dense_4_accuracy: 0.9781 - dense_5_accuracy: 0.9788 - val_loss: 0.5312 - val_dense_3_loss: 0.3562 - val_dense_4_loss: 0.0843 - val_dense_5_loss: 0.0909 - val_dense_3_accuracy: 0.9193 - val_dense_4_accuracy: 0.9803 - val_dense_5_accuracy: 0.9799
Epoch 3/10
45189/45189 [==============================] - 77s 2ms/step - loss: 0.4363 - dense

In [62]:
model.save('/kaggle/working/model_final.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model=load_model('/kaggle/working/model.h5')